# Assingment 2

What is the price of a floating strike lookback call option on a non-dividend-paying stock when the stock price is $\$52$, the risk-free interest rate is $12\%$ per annum, the volatility is $30\%$ per annum, the time to maturity is $3$ months?

- Write a Monte-Carlo simulation in Python using geometric Brownian motion as the underlying stochastic process, with and without using antithetic variates. Estimate the option price using a time grid of 1 day and 100000 trajectories. Estimate the Monte Carlo error.

Use Python and send your Jupyter notebook containing your results.

## Solution

A stochastic process $S_{t}$ is said to follow a geometric Brownian motion (GMB) if it satisfies the following stochastic differential equation:

$$
d S_{t}
=
\sigma S_{t} d W_{t} + \mu S_{t} dt
$$

Where the diffusion constant, or volatility $\sigma$ and the drift $\mu$ are constants. $W_{t}$ is the Wiener process or the Brownian motion. For an arbitrary initial value $S_{0}$ the above equation has the analytic solution

$$
S_{t}
=
S_{0} \cdot \exp{\left( \left( \mu - \frac{\sigma^{2}}{2} \right) dt + \sigma W_{t} \right)}
\equiv
S_{0} \cdot \exp{\left( \left( r - \frac{\sigma^{2}}{2} \right) \left( T - t \right) + \sigma W_{t} \right)}
$$

In the context of the financial assignment, the notation is the following:
- $S_t$ is the spot price of the underlying asset
- $T - t$ is the time to maturity (expressed in years)
- $r \equiv \mu$ is the risk free rate (annual rate, expressed in terms of continuous compounding)
- $\sigma$ is the volatility of returns of the underlying asset

The $W_{t}$ process is interpeted here as a Brownian motion, dependent of the standard normal distribution:

$$
W_{t}
=
\sqrt{T - t} \cdot \epsilon
$$

Where $\epsilon$ is a random normal variable.

#### Sources
- https://github.com/yhilpisch/py4fi/blob/master/jupyter36/11_Statistics_a.ipynb
- https://en.wikipedia.org/wiki/Geometric_Brownian_motion

In [ ]:
import numpy as np
from scipy.stats import norm

import seaborn as sns
import matplotlib as mpl
import matplotlib.cm as cm
import matplotlib.pyplot as plt

#### Just some matplotlib and seaborn parameter tuning

In [ ]:
# Set axtick dimensions
major_size = 6
major_width = 1.2
minor_size = 3
minor_width = 1
mpl.rcParams['xtick.major.size'] = major_size
mpl.rcParams['xtick.major.width'] = major_width
mpl.rcParams['xtick.minor.size'] = minor_size
mpl.rcParams['xtick.minor.width'] = minor_width
mpl.rcParams['ytick.major.size'] = major_size
mpl.rcParams['ytick.major.width'] = major_width
mpl.rcParams['ytick.minor.size'] = minor_size
mpl.rcParams['ytick.minor.width'] = minor_width

# Seaborn style settings
sns.set_style({'axes.axisbelow': True,
               'axes.edgecolor': '.1',
               'axes.facecolor': 'white',
               'axes.grid': True,
               'axes.labelcolor': '.15',
               'axes.spines.bottom': True,
               'axes.spines.left': True,
               'axes.spines.right': True,
               'axes.spines.top': True,
               'figure.facecolor': 'white',
               'font.family': ['sans-serif'],
               'font.sans-serif': ['Arial',
                'DejaVu Sans',
                'Liberation Sans',
                'Bitstream Vera Sans',
                'sans-serif'],
               'grid.color': '.8',
               'grid.linestyle': '--',
               'image.cmap': 'rocket',
               'lines.solid_capstyle': 'round',
               'patch.edgecolor': 'w',
               'patch.force_edgecolor': True,
               'text.color': '.15',
               'xtick.bottom': True,
               'xtick.color': '.15',
               'xtick.direction': 'in',
               'xtick.top': True,
               'ytick.color': '.15',
               'ytick.direction': 'in',
               'ytick.left': True,
               'ytick.right': True})

In the book Yves Hilpisch - Python for Finance, in chapter 11 there is a simple Python code proposed to implement the path generation of the geometric Brownian motion. The code is the following:

In [ ]:
def gen_paths(S_0, r, sigma, T, N, i):
    ''' Generate Monte Carlo paths for geometric Brownian motion.
    
    Parameters
    ==========
    S_0 : float
        initial stock/index value
    r : float
        constant short rate
    sigma : float
        constant volatility
    T : float
        final time horizon
    N : int
        number of time steps/intervals
    i : int
        number of paths to be simulated
        
    Returns
    =======
    paths : ndarray, shape (N + 1, i)
        simulated paths given the parameters
    '''
    paths = np.zeros((N + 1, i))
    
    # Calculate the time step
    dt = T / N
    
    # First value should be our starting S_0 price
    paths[0] = S_0
    
    for t in range(1, N + 1):
        # Choose 'i' number of random number from the standard normal distribution
        eps = np.random.standard_normal(i)
        # Optionally we can normalize these raw values as the book does
        # Here we are calculating the 
        eps = (eps - eps.mean()) / eps.std()

        paths[t] = paths[t - 1] * np.exp((r - 1/2 * sigma**2) * dt + sigma * np.sqrt(dt) * eps)
    
    return paths

### Generate and visualize paths

In [ ]:
S_0 = 52
K = 50
T = 3/12
r = 0.12
sigma = 0.3
N = int(365 * T)  # Let us consider approx. daily updates
i = 100000
S_t = gen_paths(S_0, r, sigma, T, N, i)

In [ ]:
nrows = 1
ncols = 1
fig, axes = plt.subplots(nrows=nrows, ncols=ncols, figsize=(ncols*15, nrows*8))

axislabelsize = 18
axisticksize = 15
axislegendsize = 18

axes.plot(S_t, lw=2.5, alpha=0.5)

axes.set_xlim(None, N)

axes.set_xlabel('$t$ [steps]', fontsize=axislabelsize)
axes.set_ylabel('Price of option [USD]', fontsize=axislabelsize)
axes.tick_params(axis='both', which='major', labelsize=axisticksize)

plt.show()

## Classical estimate

In [ ]:
S_t_exp = np.mean(S_t[-1])
S_t_std = np.std(S_t[-1])
print(('With dt = {0:.3f} days and {1} number of trajetories, we estimate the' +
       ' S_t price to be S_t = {2:.3f} USD +- {3:.3f} USD.').format(365 * T/N, i, S_t_exp, S_t_std))
print('Where the error of the simulation is approximately {0:.3f} USD'.format(S_t_std))

In [ ]:
S_t_var = np.var(S_t[-1])
print('Variance with the classical estimate is {0:.3f} USD^2.'. format(S_t_var))

In [ ]:
# Apply the payoff formula of the floating strike lookback call option for all of the end points
# payoff = max(S_t - S_min, 0)
payoffs = [np.max([s_t - np.min(S_t[:,idx]), 0]) for idx, s_t in enumerate(S_t[-1])]
# Estimated payoff will be average of these
payoffs_avg = np.mean(payoffs)
print('The payoff is estimated to be {0:.3f} USD'.format(payoffs_avg))

## Variance reduction with antithetic variates 

In [ ]:
# Our anthitetic estimator should have normal distribution
# with mean, E[S_t] and standard deviation sigma[S_t], which
# values are calculated in the classical example
S_anti = np.random.normal(loc=S_t_exp, scale=S_t_std, size=len(S_t[-1]))

# Combined estimator
S_comb = (S_t[-1] + S_anti) / 2

In [ ]:
S_t_exp_comb = np.mean(S_comb)
S_t_std_comb = np.std(S_comb)
print(('With dt = {0:.3f} days and {1} number of trajetories, we estimate the' +
       ' S_t price to be S_t = {2:.3f} USD +- {3:.3f} USD.').format(365 * T/N, i, S_t_exp_comb, S_t_std_comb))
print('Where the error of the simulation is approximately {0:.3f} USD'.format(S_t_std_comb))

In [ ]:
S_t_var_comb = np.var(S_comb)
print('Variance with the antithetic estimate is {0:.3f} USD^2.'. format(S_t_var_comb))

Corrected the line `np.max(s_t - np.min(S_t[:,idx])` to `np.max([s_t - np.min(S_t[:,idx], 0])`

In [ ]:
# Apply the payoff formula of the floating strike lookback call option for all of the end points
# payoff = max(S_t - S_min, 0)
payoffs_comb = [np.max([s_t - np.min(S_t[:,idx]), 0]) for idx, s_t in enumerate(S_comb)]
# Estimated payoff will be average of these
payoffs_avg_comb = np.mean(payoffs_comb)
print('The payoff is estimated to be {0:.3f} USD'.format(payoffs_avg_comb))